# Imports

In [ ]:
import traceback
import warnings
from functools import partial
from importlib import reload
from operator import getitem

import cachetools
import dask
import holoplot.pandas
import holoviews as hv
import ipywidgets as widgets
import matplotlib.pyplot as plt
import nd2reader
import numpy as np
import pandas as pd
import param
import parambokeh
import qgrid
import scipy
import skimage.morphology
import streamz
import zarr
from bokeh.models.tools import HoverTool
from cytoolz import compose, get_in
from dask import delayed
from dask.distributed import Client, LocalCluster, progress
from dask_jobqueue import SLURMCluster
from holoviews.operation.datashader import regrid
from holoviews.streams import Stream, param
from tqdm import tnrange, tqdm, tqdm_notebook
from traitlets import All

IDX = pd.IndexSlice

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# from processing import *
# from trench_detection import *
# from trench_segmentation import *
# from trench_segmentation.watershed import *
# from util import *
# from ui import *
import common
import diagnostics
import image
import metadata
import trench_detection
import ui
import util
import workflow

In [ ]:
%load_ext line_profiler
hv.extension("bokeh")
%matplotlib inline
tqdm.monitor_interval = 0

# Restore data

In [ ]:
%store -r trench_df4

In [ ]:
trench_df = trench_df4

# Config

In [ ]:
cluster = SLURMCluster(
    queue="short",
    walltime="5:00:00",
    # job_extra=['-p transfer'],
    # job_extra=['--cores-per-socket=8'],
    # interface='ib0',
    memory="64GB",
    local_directory="/tmp",
    threads=1,
    processes=1,
    # diagnostics_port=('127.0.0.1', 8787),
    env_extra=['export PYTHONPATH="/home/jqs1/projects/matriarch"'],
)
client = Client(cluster)

In [ ]:
cluster._widget().children[1].children[1].children[0].children[0].layout.width = "200px"
cluster

In [ ]:
client = Client()

In [ ]:
cluster.stop_workers(cluster.jobs)

# Functions

# Debugging

# Loading data

In [ ]:
# nd2_filenames = ['/n/scratch2/jqs1/fidelity/all/180405_txnerr.nd2', '/n/scratch2/jqs1/fidelity/all/180405_txnerr001.nd2']
# nd2_filenames = ['/n/scratch2/jqs1/fidelity/all/180405_txnerr002.nd2']#, '/n/scratch2/jqs1/fidelity/all/TrErr002_Exp.nd2']
# nd2_filenames = ['/n/scratch2/jqs1/fidelity/all/TrErr002_Exp.nd2']
nd2_filenames = [
    "/n/scratch2/jqs1/fidelity/all/180405_txnerr.nd2",
    "/n/scratch2/jqs1/fidelity/all/180405_txnerr001.nd2",
    "/n/scratch2/jqs1/fidelity/all/180405_txnerr002.nd2",
    "/n/scratch2/jqs1/fidelity/all/TrErr002_Exp.nd2",
]
# nd2_filenames = ['/home/jqs1/scratch/fidelity/180518_triplegrowthcurve/PHASE_GC001.nd2', '/home/jqs1/scratch/fidelity/180518_triplegrowthcurve/PHASE_GC002.nd2']

In [ ]:
reload(workflow)

In [ ]:
all_frames, metadata, parsed_metadata = workflow.get_nd2_frame_list(nd2_filenames)

# Reload

In [ ]:
def do_reload():
    from importlib import reload

    import diagnostics
    import trench_detection
    import util

    reload(util)
    reload(trench_detection)
    reload(diagnostics)


client.run(do_reload)
do_reload()

# Finding trenches

In [ ]:
frames_to_process = all_frames.loc[IDX[:, :200, ["BF", "MCHERRY"], :1], :]

In [ ]:
len(frames_to_process)

In [ ]:
trench_data = {
    all_frames.index[row.Index]: client.submit(
        trench_detection.get_trenches_diag,
        client.submit(
            workflow._get_nd2_frame,
            row.filename,
            row.position,
            row.channel,
            row.t,
            memmap=False,
        ),
        find_angle_setwise=True,
    )
    for row in all_frames.head().reset_index().itertuples()
}

In [ ]:
client.cancel(trench_data)

In [ ]:
trench_rows = util.apply_map_futures(
    partial(client.map, diagnostics.wrapped_diagnostics_to_dataframe), trench_data
)

In [ ]:
trench_rows_combined = util.apply_map_futures(
    client.gather, trench_rows, predicate=lambda x: x.status == "finished"
)

## Analysis

In [ ]:
trench_df = util.map_collections(
    partial(pd.concat, axis=0, sort=True), trench_rows_combined, max_level=4
)
trench_df.index = trench_df.index.droplevel(-2)
trench_df.index.names = ["filename", "position", "channel", "t", "trench_set"]

In [ ]:
trench_df = util.multi_join(trench_df, nd2_positions)

In [ ]:
trench_df = util.multi_join(trench_df, channels_to_idx)

In [ ]:
bad_angle = trench_df["trench_rotation.hough_2.angle"].abs() > 2
bad_angle.sum()

In [ ]:
bad_period = (trench_df["trench_anchors.periodogram_2.period"] - 24).abs() > 2
bad_period.sum()

In [ ]:
(bad_angle | bad_period).sum()

In [ ]:
len(trench_df)

In [ ]:
num_ts = (
    trench_df.reset_index()
    .groupby(["filename", "position", "channel"])
    .agg({"t": "count"})
    .rename(columns={"t": "num_ts"})
)

In [ ]:
selected = trench_df[bad_period]

In [ ]:
num_selected_ts = (
    selected.reset_index()
    .groupby(["filename", "position", "channel"])
    .agg({"t": "count"})
    .rename(columns={"t": "num_selected_ts"})
)

In [ ]:
trench_df = util.multi_join(util.multi_join(trench_df, num_ts), num_selected_ts)

In [ ]:
# frame_stream.event(_df=selected.index.to_frame(index=False))

# Prototyping

In [ ]:
FrameStream = ui.DataframeStream.define(
    "FrameStream", selected.index.to_frame(index=False)
)
frame_stream = FrameStream()

box = ui.dataframe_browser(frame_stream)
frame_stream.event()
box

In [ ]:
ui.show_frame_info(trench_df, frame_stream)

In [ ]:
# g = ui.show_grid(df, stream=frame_stream)
# g

In [ ]:
ui.image_viewer(frame_stream)

In [ ]:
reload(image)
reload(trench_detection)

In [ ]:
%%time
frame = workflow.get_nd2_frame(**dict(frame_stream.get_param_values()))
_, diag, _ = diagnostics.wrap_diagnostics(trench_detection.get_trenches)(frame)

In [ ]:
ui.show_plot_browser(diag, "label_1.trench_anchors")

In [ ]:
y = get_in(
    "label_1.trench_anchors.trench_anchor_profile".split("."), diag
).Curve.I.data[:, 1]

In [ ]:
plt.plot(y)

In [ ]:
f, Pxx = scipy.signal.periodogram(y)

In [ ]:
plt.plot(f, Pxx)

In [ ]:
idx = Pxx.argmax()

In [ ]:
f[idx]

In [ ]:
1 / f[idx]

In [ ]:
diag["label_1"]["trench_rotation"]["hough_2"].keys()

In [ ]:
z = get_in("label_1.trench_rotation.hough_2.log_hough".split("."), diag).data

In [ ]:
z2 = z[:, z.shape[1] // 2]

In [ ]:
plt.plot(np.trim_zeros(z2))

In [ ]:
f2, Pxx2 = scipy.signal.periodogram(np.trim_zeros(z2))

In [ ]:
hv.Curve(zip(f2, Pxx2))

In [ ]:
idx2 = Pxx2.argmax()

In [ ]:
1 / f2[idx2]

# Low-frequency components

In [ ]:
frame1 = workflow.get_nd2_frame(**dict(frame_stream.get_param_values()))

In [ ]:
plt.imshow(frame1)

In [ ]:
reload(image)

In [ ]:
def psd2(img):
    return np.abs(np.fft.fftshift(np.fft.fft2(img))) ** 2

In [ ]:
a = psd2(frame1)
b = a / a.mean()

In [ ]:
plt.plot(image.radial_profile(np.log(psd2(frame1))))

In [ ]:
plt.plot(np.log(image.radial_profile(psd2(frame1))))
plt.plot(np.log(image.radial_profile(psd2(frame2))))

In [ ]:
plt.imshow(np.log(b))

In [ ]:
plt.imshow(psd2(frame1))

In [ ]:
plt.plot(image.radial_profile(frame1))

# Image processing prototyping

In [ ]:
reload(trench_detection)

In [ ]:
%%time
_, diag2, _ = diagnostics.wrap_diagnostics(trench_detection.get_trenches)(
    workflow.get_nd2_frame(**dict(frame_stream.get_param_values()))
)

In [ ]:
ui.show_plot_browser(diag2, "labeling")

In [ ]:
a = get_in("labeling.components".split("."), diag)

In [ ]:
b = skimage.morphology.label(a.data)

# Old

In [ ]:
get_trenches(root_group["raw"][str(pos)][1, 30], diagnostics=diag_pos[pos])

In [ ]:
diag = tree()
_ = get_trenches(root_group["raw"][str(pos)][0, 1], diagnostics=diag)

In [ ]:
def f(img_stack):
    ary = np.stack(
        [
            segment_trench(img_stack[t], diagnostics=None)
            for t in range(img_stack.shape[0])
        ],
        axis=0,
    )
    ary = zarr.array(ary, compressor=DEFAULT_FRAME_COMPRESSOR)
    return ary


trench_seg_masks = positionwise_trenchwise_map(
    root_group["raw"],
    trench_points_pos,
    f,
    channel_slice=1,
    preload=True,
    time_slice=slice(None),
    positions=range(1),
)

In [ ]:
def f(img_stack):
    return pd.Series(np.percentile(img_stack, 95, axis=(1, 2)))
    # return pd.Series(np.max(img_stack, axis=(1,2)))


trench_traces_all = positionwise_trenchwise_map(
    root_group["raw"],
    trench_points_pos,
    f,
    channel_slice=2,
    preload=True,
    time_slice=slice(None),
    positions=range(100),
)